<a href="https://colab.research.google.com/github/rizkyprofs/ML_DL/blob/main/Deep_Learning_Infrared_(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deep Learning Model for Infrared Dataset
Notebook ini mencakup model regresi menggunakan PyTorch dan TensorFlow, serta evaluasi menggunakan RMSE, MSE, dan R².


In [11]:
from google.colab import files
uploaded = files.upload()

Saving secondhanddataset.csv to secondhanddataset.csv


In [14]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
file_path = "secondhanddataset.csv"
df = pd.read_csv(file_path)

# Assume 'current price' is the target and other columns are features
target_column = "current price"
X = df.drop(columns=[target_column])
y = df[target_column]

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

# Define MLP model in PyTorch
class MLP_PyTorch(nn.Module):
    def __init__(self, input_dim):
        super(MLP_PyTorch, self).__init__()
        self.fc1 = nn.Linear(input_dim, 64)
        self.fc2 = nn.Linear(64, 32)
        self.fc3 = nn.Linear(32, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Initialize model
input_dim = X_train.shape[1]
pytorch_model = MLP_PyTorch(input_dim)

# Loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(pytorch_model.parameters(), lr=0.001)

# Training loop
for epoch in range(100):
    optimizer.zero_grad()
    outputs = pytorch_model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

# Predictions
with torch.no_grad():
    y_pred_train = pytorch_model(X_train_tensor).numpy()
    y_pred_test = pytorch_model(X_test_tensor).numpy()

# Evaluation
mse = mean_squared_error(y_test, y_pred_test)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred_test)

# TensorFlow MLP model
tf_model = keras.Sequential([
    keras.layers.Dense(64, activation='relu', input_shape=(input_dim,)),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1)
])

# Compile model
tf_model.compile(optimizer='adam', loss='mse')

# Train model
tf_model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=0)

# Predictions
y_pred_test_tf = tf_model.predict(X_test)

# Evaluation
mse_tf = mean_squared_error(y_test, y_pred_test_tf)
rmse_tf = np.sqrt(mse_tf)
r2_tf = r2_score(y_test, y_pred_test_tf)

print(f'TensorFlow Model - RMSE: {rmse_tf}, MSE: {mse_tf}, R2: {r2_tf}')




/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step
TensorFlow Model - RMSE: 55308.73061997225, MSE: 3059055682.7926564, R2: 0.8180322042740477



## Penjelasan Matematika

### 1. Mean Squared Error (MSE)
\[ MSE = \frac{1}{n} \sum_{i=1}^{n} (y_i - \hat{y}_i)^2 \]
Di mana:
- \( y_i \) adalah nilai aktual,
- \( \hat{y}_i \) adalah nilai prediksi,
- \( n \) adalah jumlah sampel.

MSE mengukur rata-rata kesalahan kuadrat antara prediksi dan nilai aktual.

### 2. Root Mean Squared Error (RMSE)
\[ RMSE = \sqrt{MSE} \]
RMSE memberikan gambaran seberapa besar kesalahan rata-rata dalam unit yang sama dengan target variabel.

### 3. R-Squared (R²)
\[ R^2 = 1 - \frac{\sum (y_i - \hat{y}_i)^2}{\sum (y_i - \bar{y})^2} \]
Di mana \( \bar{y} \) adalah rata-rata nilai aktual.

R² mengukur seberapa baik model menjelaskan variasi dalam data.
